<a href="https://colab.research.google.com/github/Naga-Harshitha/Audio-Classification-using-CNN/blob/main/Prediction_of_COVID_19_Diease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_data = ImageDataGenerator(rescale=1./125)

In [6]:
val_data = ImageDataGenerator(rescale=1./125)

In [7]:
train_generator = train_data.flow_from_directory(
    '/content/gdrive/MyDrive/COVID-19 Dataset/Train',
    target_size=(150,150),
    batch_size=512,
    class_mode="binary"
)

Found 1633 images belonging to 2 classes.


In [8]:
val_generator = val_data.flow_from_directory(
    '/content/gdrive/MyDrive/COVID-19 Dataset/Val',
    target_size=(150,150),
    batch_size=512,
    class_mode="binary"
)

Found 890 images belonging to 2 classes.


In [9]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop

In [10]:
from keras.engine import sequential
model=Sequential()

In [11]:
import tensorflow as tf

In [23]:
def create_model():

  model = tf.keras.models.Sequential([ 
    tf.keras.layers.Conv2D(16,(3,3), activation = 'relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  
  model.compile(optimizer=RMSprop(lr=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy']) 
    
  return model

In [24]:
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 64)      

In [25]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('val_accuracy')>=0.90):
            self.model.stop_training = True
            
callback = mycallback()

In [28]:
history = model.fit(
    train_generator,
    epochs = 10,
    validation_data = val_generator,
)

Epoch 1/10
4/4 [==============================] - 56s 18s/step - loss: 0.6961 - accuracy: 0.6381 - val_loss: 0.6619 - val_accuracy: 0.5798
Epoch 2/10
4/4 [==============================] - 54s 13s/step - loss: 0.6769 - accuracy: 0.5413 - val_loss: 0.6564 - val_accuracy: 0.6157
Epoch 3/10
4/4 [==============================] - 53s 12s/step - loss: 0.6203 - accuracy: 0.7532 - val_loss: 0.8121 - val_accuracy: 0.5180
Epoch 4/10
4/4 [==============================] - 62s 16s/step - loss: 0.6809 - accuracy: 0.6062 - val_loss: 0.6280 - val_accuracy: 0.6820
Epoch 5/10
4/4 [==============================] - 63s 20s/step - loss: 0.6491 - accuracy: 0.5609 - val_loss: 0.6283 - val_accuracy: 0.6933
Epoch 6/10
4/4 [==============================] - 62s 19s/step - loss: 0.4963 - accuracy: 0.8408 - val_loss: 1.8572 - val_accuracy: 0.4865
Epoch 7/10
4/4 [==============================] - 54s 17s/step - loss: 0.7280 - accuracy: 0.7336 - val_loss: 0.6323 - val_accuracy: 0.6607
Epoch 8/10
4/4 [===========

In [32]:
from google.colab import files
from tensorflow.keras.preprocessing import image
uploaded=files.upload()
for i in uploaded.keys():
  path='/content/' + i
  img=image.load_img(path,target_size=(150,150))
  x=image.img_to_array(img)
  x=np.expand_dims(x,axis=0)
  images=np.vstack([x])
  classes=model.predict(images,batch_size=1)
  print(classes[0])
  if classes<0.5:
    print("COVID-19")
  else:
    print("non COVID-19")

Saving Non-Covid (746).png to Non-Covid (746) (1).png
1/1 [==============================] - 0s 29ms/step
[1.]
non COVID-19


In [31]:
model.save('data.h5')